In [ ]:
import pandas as pd
import numpy as np

In [ ]:
other_list = ["gaspe", "maritime", "newfoundland", "st_john", "st_lawrence"]

def collectdata(anylist):
    
    for mylist in anylist:
        mydf = pd.read_csv("data/" + mylist + ".csv", encoding='latin-1')
        mydf["LOCATION"] = str(mylist)
        mydf["DATE_TIME_HEURE"] = pd.to_datetime(mydf["DATE_TIME_HEURE"])
        mydf["YEAR"] = pd.DatetimeIndex(mydf["DATE_TIME_HEURE"]).year
        mydf = mydf[["SITE_NO", "YEAR", "LOCATION", "VARIABLE", "VALUE_VALEUR", "UNIT_UNITE"]]
        mydf1 = mydf.groupby(["YEAR", "LOCATION", "SITE_NO", "VARIABLE", "UNIT_UNITE"])["VALUE_VALEUR"].mean()
        mydf1 = mydf1.to_frame().reset_index()
        cols = pd.DataFrame(columns = ["SITE_NO", "LOCATION", "YEAR", "VARIABLE", "VALUE_VALEUR", "UNIT_UNITE"])
        df = pd.concat([cols, mydf1])
    return df

In [ ]:
sample = collectdata(["arctic", "assiniboine", "churchill", "columbia", "fraser",
                     "keewatin", "mackenzie", "missouri", "south_saskatchewan",
                     "north_saskatchewan", "okanagan", "pacific_coastal",
                      "athabasca", "lower_saskatchewan", "winnipeg", "yukon"])
sample

In [ ]:
sites = pd.read_csv("data/site_lists.csv", encoding='latin-1')
comb = pd.merge(sample, sites, on="SITE_NO", how="inner")
comb = comb.rename(columns={"VALUE_VALEUR":"VALUE"})
comb.head()

In [ ]:
comb["VARIABLE"].unique()

In [ ]:
variables_selected = ["PH", "OXYGEN DISSOLVED", "TEMPERATURE WATER", "PHOSPHORUS TOTAL DISSOLVED",
                     "SULPHATE DISSOLVED", "CHLORIDE"]
comb_filtered = comb[comb.VARIABLE.isin(variables_selected)]
# comb_filtered["YEAR"].astype("str")
# comb_filtered["YEAR"] = comb_filtered["YEAR"].dt.year
comb_filtered = comb_filtered.sort_values("YEAR", ascending=True)
comb_filtered.head()

In [ ]:
comb_filtered["YEAR"].unique()

In [ ]:
from plotly import tools
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import IPython.display
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

output = widgets.Output()
variables = widgets.Dropdown(options=sorted(comb_filtered.VARIABLE.unique()), value=None, description="Variable: ")

        
def water_quality_variable(change):
    comb3 = comb_filtered[comb_filtered["VARIABLE"] == variables.value]
    fig = px.scatter_mapbox(comb3,
                           lat="LATITUDE",
                           lon="LONGITUDE",
                           hover_name="PROV_TERR",
                           color="LOCATION",
                           animation_frame="YEAR",
                           size=comb3["VALUE"].abs().to_list(),
                            size_max=50,
                           opacity=0.5).update_layout(mapbox={"style": "carto-positron", "zoom": 4}, margin={"t":0,"b":0,"l":0,"r":0})
    
    customdata = np.stack((comb3["PROV_TERR"], comb3["LOCATION"], comb3["VARIABLE"], comb3["VALUE"],
                      comb3["LONGITUDE"], comb3["LATITUDE"]), axis=1)
    
    hovertemplate="<br>".join([
    "<b>PROVINCE:  %{customdata[0]}</b>",
    "<br>LONGITUDE: %{customdata[4]}",
    "LATITUDE: %{customdata[5]}",
    "LOCATION: %{customdata[1]}",
    "VARIABLE: %{customdata[2]}",
    "VALUE: %{customdata[3]}"])
    
    fig.update_traces(customdata=customdata, hovertemplate=hovertemplate)
    display(variables)
    fig.show()
    IPython.display.clear_output(wait=True)

variables.observe(water_quality_variable, names="value")
IPython.display.clear_output(wait=True)    

In [ ]:
water_quality_variable("")

In [ ]:
comb_filtered["VALUE"].max()